In [1]:
import itertools
import numpy as np
import plotly.graph_objects as go
from dash import Dash, dcc, html, Output, Input

# Definindo o tamanho da grade 4D
tamanho = 3

# Gerando todas as possíveis direções em 4D (excluindo a direção nula)
direcoes = list(itertools.product([-1, 0, 1], repeat=4))
direcoes = [d for d in direcoes if any(c != 0 for c in d)]

# Função para verificar se uma linha está dentro dos limites da grade
def linha_valida(start, direcao, tamanho):
    linha = []
    for n in range(tamanho):
        ponto = tuple(start[i] + direcao[i]*n for i in range(4))
        # Alteração: A verificação abaixo foi corrigida para incluir também pontos no limite.
        if not all(-1 <= coord < tamanho for coord in ponto):  # Permitir -1 para incluir pontos que começam fora da grade
            return None
        linha.append(ponto)
    return linha

# Coletando todas as linhas colineares sem duplicatas
linhas = []
for direcao in direcoes:
    for start in itertools.product(range(tamanho), repeat=4):
        linha = linha_valida(start, direcao, tamanho)
        if linha:
            # Alteração: Verificar se a linha ainda está dentro dos limites e adicionar
            linhas.append(linha)

print(f"Total de linhas colineares encontradas: {len(linhas)}")

# Definindo uma paleta de cores para cada valor de W
cores_w = {
    0: 'red',
    1: 'green',
    2: 'blue',
    3: 'orange',  # Adicione mais cores se tamanho > 3
    4: 'purple',
    5: 'cyan',
    6: 'magenta',
    7: 'yellow',
    8: 'brown',
    # Continue conforme necessário
}

# Função para calcular o deslocamento baseado em W
def calcular_deslocamento(w, tamanho):
    # Você pode ajustar o eixo e o fator de deslocamento conforme necessário
    deslocamentos = {
        'x': (0, 0, 0),
        'y': (w * 3,0, 0),
        'z': (0, w * 1.5, 0)
    }
    # Escolha um eixo para deslocar com base em W
    # Por exemplo, alternar entre eixos para diferentes W
    eixo = list(deslocamentos.keys())[w % 3]
    return deslocamentos[eixo]

# Função de projeção isométrica de 4D para 3D
def isometric_projection(point4d):
    x, y, z, w = point4d
    angle = np.radians(35.264)  # Aproximação para isometria 4D
    
    cos_angle = np.cos(angle)
    sin_angle = np.sin(angle)
    
    # Rotação no plano XW
    x_rot = x * cos_angle - w * sin_angle
    w_rot = x * sin_angle + w * cos_angle
    
    # Rotação no plano YW
    y_rot = y * cos_angle - w_rot * sin_angle
    w_rot = y * sin_angle + w_rot * cos_angle
    
    # Rotação no plano ZW
    z_rot = z * cos_angle - w_rot * sin_angle
    w_rot = z * sin_angle + w_rot * cos_angle
    
    # Projeção isométrica 4D para 3D (ignorando a dimensão W após a rotação)
    x_proj = x_rot
    y_proj = y_rot
    z_proj = z_rot  # Agora, z_proj é a coordenada Z rotacionada
    
    return (x_proj, y_proj, z_proj)


# Inicializando a aplicação Dash
app = Dash(__name__)

# Layout da aplicação
app.layout = html.Div([
    html.H1("Linhas Colineares do Jogo da Velha em 4D"),
    html.Div([
        html.Label("Filtrar por X:"),
        dcc.Dropdown(
            id='filtro-x',
            options=[{'label': f'X={i}', 'value': i} for i in range(-1, tamanho)] + [{'label': 'Todos', 'value': 'all'}],
            value='all',
            multi=False
        )
    ], style={'width': '20%', 'display': 'inline-block'}),
    
    html.Div([
        html.Label("Filtrar por Y:"),
        dcc.Dropdown(
            id='filtro-y',
            options=[{'label': f'Y={i}', 'value': i} for i in range(-1, tamanho)] + [{'label': 'Todos', 'value': 'all'}],
            value='all',
            multi=False
        )
    ], style={'width': '20%', 'display': 'inline-block', 'marginLeft': '2%'}),
    
    html.Div([
        html.Label("Filtrar por Z:"),
        dcc.Dropdown(
            id='filtro-z',
            options=[{'label': f'Z={i}', 'value': i} for i in range(-1, tamanho)] + [{'label': 'Todos', 'value': 'all'}],
            value='all',
            multi=False
        )
    ], style={'width': '20%', 'display': 'inline-block', 'marginLeft': '2%'}),
    
    html.Div([
        html.Label("Filtrar por W:"),
        dcc.Dropdown(
            id='filtro-w',
            options=[{'label': f'W={i}', 'value': i} for i in range(-1, tamanho)] + [{'label': 'Todos', 'value': 'all'}],
            value='all',
            multi=False
        )
    ], style={'width': '20%', 'display': 'inline-block', 'marginLeft': '2%'}),
    
    dcc.Graph(id='grafico-3d', style={'height': '800px'})
])

# Atualizando a função de callback para incluir a correção de deslocamento na direção
@app.callback(
    Output('grafico-3d', 'figure'),
    Input('filtro-x', 'value'),
    Input('filtro-y', 'value'),
    Input('filtro-z', 'value'),
    Input('filtro-w', 'value')
)
def atualizar_grafico(filtro_x, filtro_y, filtro_z, filtro_w):
    # Filtrar as linhas com base nos filtros selecionados
    linhas_filtradas = []
    for linha in linhas:
        x_val = linha[0][0]
        y_val = linha[0][1]
        z_val = linha[0][2]
        w_val = linha[0][3]

        # Aplicar os filtros corretamente:
        # Se o filtro for 'all', ignoramos esse eixo.
        # Se o filtro for -1, incluímos apenas as linhas que têm -1 nessa dimensão.
        if ((filtro_x == 'all' or x_val == filtro_x or (filtro_x == -1 and any(p[0] == -1 for p in linha))) and
            (filtro_y == 'all' or y_val == filtro_y or (filtro_y == -1 and any(p[1] == -1 for p in linha))) and
            (filtro_z == 'all' or z_val == filtro_z or (filtro_z == -1 and any(p[2] == -1 for p in linha))) and
            (filtro_w == 'all' or w_val == filtro_w or (filtro_w == -1 and any(p[3] == -1 for p in linha)))):
            linhas_filtradas.append(linha)

    # Preparar os traces para o Plotly (resto do código permanece igual)
    traces = []
    for linha in linhas_filtradas:
        x = [p[0] for p in linha]
        y = [p[1] for p in linha]
        z = [p[2] for p in linha]
        w = linha[0][3]  # Usando a coordenada w do ponto inicial para a cor
        cor = cores_w.get(w, 'black')  # Default para 'black' se w não estiver em cores_w

        # Calcular deslocamento
        desloc_x, desloc_y, desloc_z = calcular_deslocamento(w, tamanho)
        x_desloc = [xi + desloc_x for xi in x]
        y_desloc = [yi + desloc_y for yi in y]
        z_desloc = [zi + desloc_z for zi in z]

        # Adicionar hovertemplate para mostrar as coordenadas originais sem deslocamento
        hovertemplate = '<b>Coordenadas originais:</b><br>' + \
                        'X: %{customdata[0]}<br>' + \
                        'Y: %{customdata[1]}<br>' + \
                        'Z: %{customdata[2]}<br>' + \
                        'W: %{customdata[3]}<br>'

        # Atualizar o nome da linha na legenda para incluir a dimensão W com o deslocamento
        traces.append(
            go.Scatter3d(
                x=x_desloc,
                y=y_desloc,
                z=z_desloc,
                mode='lines',
                line=dict(color=cor, width=5),
                opacity=0.8,
                customdata=np.array(linha),  # Passa as coordenadas originais como customdata
                hovertemplate=hovertemplate,
                name=f'Direção: {linha[0]}',  # Atualizando a legenda com W e deslocamento
                showlegend=True
            )
        )
    
    # Definir a legenda única para cores
    unique_ws = sorted(set([linha[0][3] for linha in linhas_filtradas]))
    legend_traces = []
    for w in unique_ws:
        if w in cores_w:
            legend_traces.append(
                go.Scatter3d(
                    x=[None],
                    y=[None],
                    z=[None],
                    mode='markers',
                    marker=dict(size=0, color=cores_w[w]),
                    name=f'W={w}'
                )
            )
    
    # Combinar os traces das linhas com os traces da legenda
    all_traces = traces + legend_traces
    
    # Criar o layout do gráfico
    layout = go.Layout(
        title='Linhas Colineares do Jogo da Velha em 4D',
        scene=dict(
            xaxis=dict(title='X', range=[-0.5-1, tamanho*2-0.5]),
            yaxis=dict(title='Y', range=[-0.5-1, tamanho*2-0.5]),
            zaxis=dict(title='Z', range=[-0.5-1, tamanho-0.5]),
        ),
        legend=dict(
            itemsizing='constant'
        ),
        width=900,
        height=800
    )
    
    fig = go.Figure(data=all_traces, layout=layout)
    
    return fig

app.run_server(debug=True)

Total de linhas colineares encontradas: 1215
